# TensorFlow Scala Example


First we have to load [TensorFlowScala](http://platanios.org/tensorflow_scala) into our kernel.

In [1]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module("org.platanios", "tensorflow_2.12"),
    version = "0.4.1",
    attributes = coursier.Attributes("", "darwin-cpu-x86_64") // replace with linux-cpu-x86_64 or linux-gpu-x86_64 on linux
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [2]:
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader

import java.nio.file.Paths
import scala.util.Random

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.learn._

import org.platanios.tensorflow.api.learn.layers._

import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator

import org.platanios.tensorflow.data.image.MNISTLoader


import java.nio.file.Paths

import scala.util.Random

In [3]:
val dataset = MNISTLoader.load(Paths.get("tmp"))

2018-11-24 14:28:15.741 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file 'tmp/train-images-idx3-ubyte.gz'.
2018-11-24 14:28:17.320 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file 'tmp/train-labels-idx1-ubyte.gz'.
2018-11-24 14:28:17.328 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file 'tmp/t10k-images-idx3-ubyte.gz'.
2018-11-24 14:28:17.394 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file 'tmp/t10k-labels-idx1-ubyte.gz'.
2018-11-24 14:28:17.396 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


dataset: org.platanios.tensorflow.data.image.MNISTDataset = MNISTDataset(
  MNIST,
  Tensor[UByte, [60000, 28, 28]],
  Tensor[UByte, [60000]],
  Tensor[UByte, [10000, 28, 28]],
  Tensor[UByte, [10000]]
)

Let's display a few example images from our training set.

In [4]:
val sess = Session()
def showImage(image: Tensor[UByte]): Unit = {
  val exampleImage = tf.decodeRaw[Byte](tf.image.encodePng(image))
  val png = sess.run(fetches = exampleImage)
  Image.fromArray(png.entriesIterator.toArray, Image.PNG, width=Some("50"))
}

val randomImages = for {
  index <- (0 to 4).map(_ => Random.nextInt(dataset.testImages.shape(0)))
  image = dataset.testImages(index).expandDims(-1)
} yield image
randomImages.foreach(showImage(_))

sess: Session = org.platanios.tensorflow.api.core.client.Session@38fbcf85
defined function showImage
randomImages: collection.immutable.IndexedSeq[Tensor[UByte]] = Vector(
  Tensor[UByte, [28, 28, 1]],
  Tensor[UByte, [28, 28, 1]],
  Tensor[UByte, [28, 28, 1]],
  Tensor[UByte, [28, 28, 1]],
  Tensor[UByte, [28, 28, 1]]
)

In [5]:
// Load and batch data using pre-fetching.
val trainImages = tf.data.datasetFromTensorSlices(dataset.trainImages.toFloat)
val trainLabels = tf.data.datasetFromTensorSlices(dataset.trainLabels.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(10000)
      .batch(256)
      .prefetch(10)

trainImages: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset]
trainLabels: ops.data.Dataset[Output[Long]] = Dataset[TensorSlicesDataset]
trainData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Zip/Repeat/Shuffle/Batch/Prefetch]

In [6]:
// Create the MLP model.
val input = Input(FLOAT32, Shape(-1, 28, 28))
val trainInput = Input(INT64, Shape(-1))
val layer = Flatten[Float]("Input/Flatten") >>
    Linear[Float]("Layer_0", 128) >> ReLU[Float]("Layer_0/Activation", 0.1f) >>
    Linear[Float]("Layer_1", 64) >> ReLU[Float]("Layer_1/Activation", 0.1f) >>
    Linear[Float]("Layer_2", 32) >> ReLU[Float]("Layer_2/Activation", 0.1f) >>
    Linear[Float]("OutputLayer", 10)
val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >>
    Mean("Loss/Mean")
val optimizer = tf.train.GradientDescent(1e-6f)
val model = Model.simpleSupervised(input, trainInput, layer, loss, optimizer)

input: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@72322be
trainInput: Input[Output[Long]] = org.platanios.tensorflow.api.learn.layers.Input@16bfec77
layer: Compose[Output[Float], Output[Float], Output[Float]] = Compose(
  "Input/Flatten",
  Compose(
    "Input/Flatten",
    Compose(
      "Input/Flatten",
      Compose(
        "Input/Flatten",
        Compose(
          "Input/Flatten",
          Compose(
            "Input/Flatten",
            Compose(
              "Input/Flatten",
              Flatten("Input/Flatten"),
              Linear(
                "Layer_0",
                128,
                true,
                RandomNormalInitializer(
                  Tensor[Float, []],
                  Tensor[Float, []],
                  None
                ),
                RandomNormalInitializer(
                  Tensor[Float, []],
                  Tensor[Float, []],
                  None
                )
              )
            ),
     

In [7]:
import org.platanios.tensorflow.api.learn.hooks._
import org.platanios.tensorflow.api.config.TensorBoardConfig

val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >>
    Mean("Loss/Mean") >>
    ScalarSummary(name = "Loss", tag = "Loss")
val summariesDir = Paths.get("/tmp/summaries")
val estimator = InMemoryEstimator(
  modelFunction = model,
  configurationBase = Configuration(Some(summariesDir)),
  trainHooks = Set(
    SummarySaver(summariesDir, StepHookTrigger(100)),
    CheckpointSaver(summariesDir, StepHookTrigger(1000))),
  tensorBoardConfig = TensorBoardConfig(summariesDir))

2018-11-24 14:38:34.854 [scala-interpreter-1] INFO  Variables / Saver - Restoring parameters from '/tmp/summaries/model.ckpt-1000'.


import org.platanios.tensorflow.api.learn.hooks._

import org.platanios.tensorflow.api.config.TensorBoardConfig


loss: Compose[(Output[Float], Output[Long]), Output[Float], Output[Float]] = Compose(
  "Loss",
  Compose(
    "Loss",
    SparseSoftmaxCrossEntropy("Loss"),
    Mean("Loss/Mean", null, false)
  ),
  ScalarSummary(
    "Loss",
    "Loss",
    null,
    Set(org.platanios.tensorflow.api.core.Graph$Keys$SUMMARIES$@17ccd9e0)
  )
)
summariesDir: java.nio.file.Path = /tmp/summaries
estimator: InMemoryEstimator[Output[Float], (Output[Float], Output[Long]), Output[Float], Output[Float], Float, (Output[Float], (Output[Float], Output[Long]))] = org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator@39294572

In [8]:
estimator.train(() => trainData, StopCriteria(maxSteps = Some(10000)))

2018-11-24 14:38:59.519 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 2000.
2018-11-24 14:38:59.520 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/summaries/model.ckpt-2000'.
2018-11-24 14:39:00.924 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/summaries/model.ckpt-2000'.
2018-11-24 14:39:05.495 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 3000.
2018-11-24 14:39:05.496 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/summaries/model.ckpt-3000'.
2018-11-24 14:39:06.369 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/tmp/summaries/model.ckpt-3000'.
2018-11-24 14:39:09.883 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 4000.
2018-11-24 14:39:09.883 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/tmp/summaries/model.ckpt-4000'.
2018-11

In [26]:
val images = dataset.testImages(0::10)
val result = estimator.infer(() => images.toFloat)
val predictedClasses = result.argmax(1)
for (i <- 0 until images.shape(0)) {
    showImage(images(i).expandDims(-1))
    print("predicted class: ")
    print(predictedClasses(i).scalar)
}

predicted class: 7

predicted class: 2

predicted class: 1

predicted class: 0

predicted class: 4

...

predicted class: 4

predicted class: 9

predicted class: 6

predicted class: 9

images: Tensor[UByte] = Tensor[UByte, [10, 28, 28]]
result: Tensor[Float] = Tensor[Float, [10, 10]]
predictedClasses: Tensor[Long] = Tensor[Long, [10]]